<a href="https://colab.research.google.com/github/jbrown544/interlingual-topic-modeling/blob/main/Interlingual_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Thematic Discovery in Non-English Natural Language Corpora

In [1]:
# install the required gensim package with optional dependencies
!pip install gensim==4.0.0
!pip install python-Levenshtein

# verify gensim package information
!pip show gensim

Name: gensim
Version: 4.0.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, smart-open, scipy
Required-by: pyLDAvis


In [2]:
# install the required pyLDAvis package
!pip install pyLDAvis==3.3.1

# verify pyLDAvis package information
!pip show pyLDAvis

Name: pyLDAvis
Version: 3.3.1
Summary: Interactive topic model visualization. Port of the R package.
Home-page: https://github.com/bmabey/pyLDAvis
Author: Ben Mabey
Author-email: ben@benmabey.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: future, sklearn, jinja2, joblib, numpy, pandas, gensim, scipy, setuptools, scikit-learn, numexpr, funcy
Required-by: 


In [3]:
# import dependencies

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from urllib.parse import urljoin

from smart_open import open

from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X

In [4]:
# samples of french language text

french_book_url = 'https://raw.githubusercontent.com/jbrown544/interlingual-topic-modeling/main/text/books/fr/'

jules_verne_french_books = [
  urljoin(french_book_url, 'Around_the_World_in_Eighty_Days.txt'),
  urljoin(french_book_url, 'Five_Weeks_in_a_Balloon.txt'),
  urljoin(french_book_url, 'From_the_Earth_to_the_Moon.txt'),
  urljoin(french_book_url, 'Robur_the_Conqueror.txt'),
  urljoin(french_book_url, 'The_Mutineers_of_the_Bounty.txt')
]

In [5]:
# prototype of preprocessing pipeline
def preprocess_pipeline(text, 
                        deaccent=False, 
                        min_token_len=2, 
                        max_token_length=15):
  
  return simple_preprocess(text, deacc=deaccent, 
                           min_len=min_token_len, 
                           max_len=max_token_length)

In [6]:
# read and process texts into token lists
jules_verne_texts = [preprocess_pipeline(open(book, encoding='latin-1').read()) for book in jules_verne_french_books]

In [7]:
# construct dictionary from token lists
jules_verne_dictionary = corpora.Dictionary(jules_verne_texts)

print(jules_verne_dictionary)

Dictionary(20937 unique tokens: ['abaissait', 'abaissant', 'abaissement', 'abaissé', 'abandonna']...)


In [8]:
# create a corpus by converting token lists into a documet-term matrix
jules_verne_corpus = [jules_verne_dictionary.doc2bow(text) for text in jules_verne_texts]

In [9]:
# construct an HDP model (topics determined during processing)
jules_verne_hdp_model = models.HdpModel(jules_verne_corpus, 
                                        id2word=jules_verne_dictionary)

In [10]:
# enable visualization inline in notebook
pyLDAvis.enable_notebook()

In [11]:
# visualize suggested LDA model generated from HDP model
gensimvis.prepare(jules_verne_hdp_model.suggested_lda_model(),
                  jules_verne_corpus, jules_verne_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0      0.000143  0.000145       1        1  3.003067e+01
1      0.000256 -0.000154       2        1  2.575404e+01
2      0.000026  0.000088       3        1  2.159191e+01
3     -0.000221  0.000080       4        1  2.001623e+01
4      0.000161 -0.000299       5        1  2.607057e+00
...         ...       ...     ...      ...           ...
145    0.000099 -0.000002     146        1  2.892766e-47
146    0.000153  0.000186     147        1  1.446383e-47
147    0.000119  0.000037     148        1  7.231916e-48
148    0.000036 -0.000013     149        1  3.615958e-48
149   -0.000099 -0.000164     150        1  3.615958e-48

[150 rows x 5 columns], topic_info=             Term          Freq      Total  Category  logprob  loglift
17017  plongeants  1.500000e+01  15.000000   Default  30.0000  30.0000
13234   préparons  1.500000e+01  15.000000   Default  29.0000  29.0000
4061       honoré  1.500000e+01  15.000000   Default  28.0000  28.0000
9354   événements  1.500000e+01  15.000000   Default  27.0000  27.0000
16716     montrés  1.500000e+01  15.000000   Default  26.0000  26.0000
...           ...           ...        ...       ...      ...      ...
6426   produisait  6.128009e-49  13.021267  Topic150  -9.6624   0.2635
5251   montraient  6.109084e-49  12.923308  Topic150  -9.6655   0.2680
9910      baobabs  6.078074e-49  12.653770  Topic150  -9.6705   0.2840
3536      fondant  6.113272e-49  13.365550  Topic150  -9.6648   0.2350
11707       guéri  6.109697e-49  13.644021  Topic150  -9.6654   0.2138

[7705 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
1          1  0.307278      abaissant
1          2  0.230458      abaissant
1          3  0.230458      abaissant
1          4  0.153639      abaissant
14854      1  0.257812  abandonnaient
...      ...       ...            ...
9357       4  0.204650          êtres
9358       1  0.245834            île
9358       2  0.245834            île
9358       3  0.245834            île
9358       4  0.245834            île

[22369 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150])